<a href="https://colab.research.google.com/github/TheLemonPig/RL-SSM/blob/main/rlwm_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random, pickle
import matplotlib.pyplot as plt
import pandas as pd
import pytensor
import pytensor.tensor as pt

###PyTensor

In [21]:
def rlwm_step(dR, pA, pG, pP, dq_RL, dq_WM):
    cond = pt.switch((dR - dq_RL) >= 0, 1, 0)
    dq_RL = dq_RL + (cond + (1.0 - cond) * pG) * pA * (dR - dq_RL)
    dq_WM = dq_WM + (cond + (1.0 - cond) * pG) * 1.0 * (dR - dq_RL)
    dq_WM = dq_WM + pP * ((1 / dR.shape[2]) - dq_WM)
    return dq_RL, dq_WM


def rlwm_scan(dR, pA, pG, pP, dq_RL, dq_WM):
    results, _ = pytensor.scan(rlwm_step, sequences=[dR, pA, pG, pP], non_sequences=[], outputs_info=[dq_RL, dq_WM])
    dq_RL, dq_WM = results
    return dq_RL, dq_WM


def rlwm_softmax(Qs, pB):
    shape = Qs.shape
    tempered_qs = Qs * pB
    qs_max = tempered_qs.max(axis=2)
    qs_max = pt.repeat(qs_max.reshape((shape[0], shape[1], 1, shape[3], shape[4])), shape[2], axis=2)
    numerator = pt.exp(tempered_qs - qs_max)
    denominator = numerator.sum(axis=2)
    denominator = pt.repeat(denominator.reshape((shape[0], shape[1], 1, shape[3], shape[4])), shape[2], axis=2)
    Ps = numerator / denominator
    return Ps


def rlwm_likelihood(dC, dq_RL, dq_WM, pB, pC, pE, pR, set_sizes):
    weight = pR * pt.clip(pC / set_sizes, 1, pt.inf)
    Ps_RL = rlwm_softmax(dq_RL, pB)
    Ps_WM = rlwm_softmax(dq_WM, pB)
    pol = weight * Ps_WM + (1.0 - weight) * Ps_RL
    pol_final = (1.0 - pE) * pol + pE * (1.0 / dC.shape[2])
    likelihood = pt.log(pol_final)
    return likelihood


def rlwm_recovery(dq_RL, dq_WM, dC, dR, pA, pB, pC, pE, pG, pP, pR, set_sizes):
    # dq_RL = pt.ones_like(dC) * 1.0 / dC.shape[2]
    # dq_WM = pt.ones_like(dC) * 1.0 / dC.shape[2]
    dq_RL, dq_WM = rlwm_scan(dR, pA, pG, pP, dq_RL, dq_WM)
    likelihood = rlwm_likelihood(dC, dq_RL, dq_WM, pB, pC, pE, pR, set_sizes)
    return likelihood


def rlwm_step_compile():
    dR4 = pt.dtensor4("dR4")
    dq_RL4 = pt.dtensor4("dq_RL4")
    dq_WM4 = pt.dtensor4("dq_WM4")
    pA4 = pt.dtensor4("pA4")
    pG4 = pt.dtensor4("pG4")
    pP4 = pt.dtensor4("pP4")

    dq_RL, dq_WM = rlwm_step(dR4, pA4, pG4, pP4, dq_RL4, dq_WM4)
    rlwm_step_func = pytensor.function(inputs=[dR4, pA4, pG4, pP4, dq_RL4, dq_WM4], outputs=[dq_RL, dq_WM])

    return rlwm_step_func


def rlwm_scan_compile():
    dR5 = pt.dtensor5("dR5")
    dq_RL5 = pt.dtensor5("dq_RL5")
    dq_WM5 = pt.dtensor5("dq_WM5")
    pA5 = pt.dtensor5("pA5")
    pG5 = pt.dtensor5("pG5")
    pP5 = pt.dtensor5("pP5")

    dq_RL, dq_WM = rlwm_scan(dR5, pA5, pG5, pP5, dq_RL5, dq_WM5)
    rlwm_step_func = pytensor.function(inputs=[dR5, pA5, pG5, pP5, dq_RL5, dq_WM5], outputs=[dq_RL, dq_WM])

    return rlwm_step_func


def rlwm_likelihood_compile():
    dC5 = pt.dtensor5("dC5")
    dq_RL5 = pt.dtensor5("dq_RL5")
    dq_WM5 = pt.dtensor5("dq_WM5")
    pB5 = pt.dtensor5("pB5")
    pC5 = pt.dtensor5("pC5")
    pE5 = pt.dtensor5("pE5")
    pR5 = pt.dtensor5("pR5")
    set_sizes = pt.dtensor5("set_sizes")

    likelihood = rlwm_likelihood(dC5, dq_RL5, dq_WM5, pB5, pC5, pE5, pR5, set_sizes)
    rlwm_likelihood_func = pytensor.function(inputs=[dC5, dq_RL5, dq_WM5, pB5, pC5, pE5, pR5, set_sizes], outputs=[likelihood])

    return rlwm_likelihood_func


def rlwm_recovery_compile():
    dq_RL = pt.dtensor5("dq_RL")
    dq_WM = pt.dtensor5("dq_WM")
    dC = pt.dtensor5("dC")
    dR = pt.dtensor5("dR")
    pA = pt.dtensor5("pA")
    pG = pt.dtensor5("pG")
    pP = pt.dtensor5("pP")
    pB = pt.dtensor5("pB")
    pC = pt.dtensor5("pC")
    pE = pt.dtensor5("pE")
    pR = pt.dtensor5("pR")
    set_sizes = pt.dtensor5("set_sizes")

    likelihood = rlwm_recovery(dq_RL, dq_WM, dC, dR, pA, pB, pC, pE, pG, pP, pR, set_sizes)
    rlwm_recovery_func = pytensor.function(inputs=[dq_RL, dq_WM, dC, dR, pA, pB, pC, pE, pG, pP, pR, set_sizes], outputs=[likelihood])

    return rlwm_recovery_func


def rlwm_step_test():
    n_participants = 6
    n_choices = 3
    n_stimuli = [4, 5, 6, 7, 8]
    max_stimuli = max(n_stimuli)
    n_blocks = 5
    shape4 = (n_participants, n_choices, max_stimuli, n_blocks)

    dR4_ = np.random.randint(low=0, high=1, size=shape4)
    dq_RL4_ = - np.ones_like(dR4_) * 1 / n_choices
    dq_WM4_ = - np.ones_like(dR4_) * 1 / n_choices
    pA4_ = np.ones_like(dR4_) * 0.1
    pG4_ = np.ones_like(dR4_) * 0.9
    pP4_ = np.ones_like(dR4_) * 1.0

    test_func = rlwm_step_compile()

    return test_func(dR4_, dq_RL4_, dq_WM4_, pA4_, pG4_, pP4_)


def rlwm_scan_test():
    n_trials = 23
    n_participants = 6
    n_choices = 3
    n_stimuli = [4, 5, 6, 7, 8]
    max_stimuli = max(n_stimuli)
    n_blocks = 5
    shape5 = (n_trials, n_participants, n_choices, max_stimuli, n_blocks)

    dR5_ = np.random.randint(low=0, high=1, size=shape5)
    dq_RL5_ = - np.ones_like(dR5_) * 1 / n_choices
    dq_WM5_ = - np.ones_like(dR5_) * 1 / n_choices
    pA5_ = np.ones_like(dR5_) * 0.1
    pG5_ = np.ones_like(dR5_) * 0.9
    pP5_ = np.ones_like(dR5_) * 1.0

    test_func = rlwm_scan_compile()

    return test_func(dR5_, dq_RL5_, dq_WM5_, pA5_, pG5_, pP5_)


def rlwm_likelihood_test():
    n_trials = 23
    n_participants = 6
    n_choices = 3
    n_stimuli = [4, 5, 6, 7, 8]
    max_stimuli = max(n_stimuli)
    n_blocks = 5
    shape5 = (n_trials, n_participants, n_choices, max_stimuli, n_blocks)

    dC5_ = np.random.randint(low=0, high=n_choices, size=shape5)
    dq_RL5_ = - np.ones_like(dC5_) * 1 / n_choices
    dq_WM5_ = - np.ones_like(dC5_) * 1 / n_choices
    pB5_ = np.ones_like(dC5_) * 1.0
    pC5_ = np.ones_like(dC5_) * 4.0
    pE5_ = np.ones_like(dC5_) * 0.5
    pR5_ = np.ones_like(dC5_) * 0.8
    set_sizes_ = np.tile(n_stimuli, (shape5[0], shape5[1], shape5[2], shape5[3], 1))

    test_func = rlwm_likelihood_compile()

    return test_func(dC5_, dq_RL5_, dq_WM5_, pB5_, pC5_, pE5_, pR5_, set_sizes_)


def rlwm_recovery_test():
    n_trials = 23
    n_participants = 7
    n_choices = 4
    n_stimuli = [4, 5, 6, 7, 8]
    max_stimuli = max(n_stimuli)
    n_blocks = 5
    shape = (n_trials, n_participants, n_choices, max_stimuli, n_blocks)

    dq_RL_ = np.ones(shape) * 1.0 / n_choices
    dq_WM_ = np.ones(shape) * 1.0 / n_choices

    dC_ = np.random.randint(low=0, high=n_choices, size=shape)
    dR_ = np.random.randint(low=0, high=1, size=shape)
    pA_ = np.ones_like(dC_) * 0.01
    pB_ = np.ones_like(dC_) * 1.0
    pC_ = np.ones_like(dC_) * 4.0
    pE_ = np.ones_like(dC_) * 0.02
    pG_ = np.ones_like(dC_) * 0.8
    pP_ = np.ones_like(dC_) * 0.3
    pR_ = np.ones_like(dC_) * 0.8
    set_sizes_ = np.tile(n_stimuli, (shape[0], shape[1], shape[2], shape[3], 1))

    test_func = rlwm_recovery_compile()

    return test_func(dq_RL_, dq_WM_, dC_, dR_, pA_, pB_, pC_, pE_, pG_, pP_, pR_, set_sizes_)

In [ ]:
rlwm_step_compile()
rlwm_step_test()

In [ ]:
rlwm_scan_compile()
rlwm_scan_test()

In [24]:
rlwm_likelihood_compile()
rlwm_likelihood_test()

[array([[[[[-1.09861229, -1.09861229, -1.09861229, -1.09861229,
            -1.09861229],
           [-1.09861229, -1.09861229, -1.09861229, -1.09861229,
            -1.09861229],
           [-1.09861229, -1.09861229, -1.09861229, -1.09861229,
            -1.09861229],
           ...,
           [-1.09861229, -1.09861229, -1.09861229, -1.09861229,
            -1.09861229],
           [-1.09861229, -1.09861229, -1.09861229, -1.09861229,
            -1.09861229],
           [-1.09861229, -1.09861229, -1.09861229, -1.09861229,
            -1.09861229]],
 
          [[-1.09861229, -1.09861229, -1.09861229, -1.09861229,
            -1.09861229],
           [-1.09861229, -1.09861229, -1.09861229, -1.09861229,
            -1.09861229],
           [-1.09861229, -1.09861229, -1.09861229, -1.09861229,
            -1.09861229],
           ...,
           [-1.09861229, -1.09861229, -1.09861229, -1.09861229,
            -1.09861229],
           [-1.09861229, -1.09861229, -1.09861229, -1.09861229,
 

In [25]:
rlwm_recovery_compile()
rlwm_recovery_test()

AssertionError: Could not broadcast dimensions. Broadcasting is only allowed along axes that have a statically known length 1. Use `specify_broadcastable` to inform PyTensor of a known shape.
Apply node that caused the error: Assert{msg=Could not broadcast dimensions. Broadcasting is only allowed along axes that have a statically known length 1. Use `specify_broadcastable` to inform PyTensor of a known shape.}(Shape_i{1}.0, Eq.0)
Toposort index: 63
Inputs types: [TensorType(int64, shape=()), TensorType(bool, shape=())]
Inputs shapes: [(), ()]
Inputs strides: [(), ()]
Inputs values: [array(7), array(False)]
Outputs clients: [[Composite{...}(Shape_i{0}.0, Shape_i{0}.0, Shape_i{0}.0, Shape_i{0}.0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, Assert{msg=Could not broadcast dimensions. Broadcasting is only allowed along axes that have a statically known length 1. Use `specify_broadcastable` to inform PyTensor of a known shape.}.0, Assert{msg=Could not broadcast dimensions. Broadcastin

###RLWM Data Generation

In [3]:
model_config_rl = {
    "RLWM_v2": {
        "doc": "RLWM model v2.",
        "params": ["rl_alpha", "rl_phi", "rl_rho", "rl_gamma", "rl_epsilon", "C"],
        "param_bounds": [[0.0, 0.0, 0.0, 0.0, 0.0, 2.0], [1.0, 1.0, 1.0, 1.0, 1.0, 5.0]],
    }
}

In [4]:
def gen_trial_seq(set_size, num_rep_per_block):
    trials = list()

    for stim in np.arange(set_size):
        trials.extend([stim]*random.choice(num_rep_per_block))
    random.shuffle(trials)

    return trials

def gen_SR_map(set_size, num_actions):
    S_R_map = {}
    acts = np.arange(num_actions)

    for stim in np.arange(set_size):
        S_R_map[stim] = random.choice(acts)

    return S_R_map

def step_action(s, a, S_R_map):
    if a == S_R_map[s]:
        rew = 1
    else:
        rew = 0

    return rew

def softmax(q_val, beta):
    q_val = np.array(q_val)*beta
    q_val = q_val - np.max(q_val)
    q_val = np.exp(q_val)
    q_val = q_val / np.sum(q_val)
    return q_val

In [5]:
def simulate_RLWM(param_rl, set_size_list, num_rep_per_block, num_actions, subjs=1):

    sub_list = np.arange(subjs)
    actions = np.arange(num_actions)

    sub_list_sub_id = list()
    sub_list_block_num = list()
    sub_list_stim = list()
    sub_list_actions = list()
    sub_list_reward = list()
    sub_list_corr = list()
    sub_list_stim_ctr = list()
    sub_list_set_size = list()

    for subj in sub_list:
        # RLWM parameters
        alpha = param_rl[0] # RL learning rate
        phi = param_rl[1] # WM decay
        rho = param_rl[2] # WM weight
        gamma = param_rl[3] # perseveration on negative outcomes
        epsilon = param_rl[4] # weight of uniform policy
        C = param_rl[5] # WM capacity
        beta = 100 # inverse temperature

        pol = np.zeros(num_actions)

        for bl in range(len(set_size_list)):
            #print(">> block -- ", bl)
            set_size = set_size_list[bl]

            S_R_map = gen_SR_map(set_size, num_actions)
            trials = gen_trial_seq(set_size, num_rep_per_block)

            # print(trials)

            q_RL = np.ones((set_size, num_actions)) * 1/num_actions
            q_WM = np.ones((set_size, num_actions)) * 1/num_actions
            weight = rho * min(1, C/set_size)

            stim_counter = np.zeros(set_size)

            for tr in np.arange(len(trials)):
                state = trials[tr]
                stim_counter[state] += 1

                pol_RL = softmax(q_RL[state, :], beta)
                pol_WM = softmax(q_WM[state, :], beta)

                pol = weight * pol_WM + (1-weight) * pol_RL

                pol_final = (1 - epsilon) * pol + epsilon * np.tile([1/num_actions], num_actions)

                action = np.random.choice(actions, p=pol_final)


                reward = step_action(state, action, S_R_map)
                #print("\t\t\tdone action", state, action, reward)

                #print("\t\t\tupdating q")
                if reward == 1:
                    sub_list_corr.append(1)
                    q_RL[state, action] = q_RL[state, action] + alpha * (reward - q_RL[state, action])
                    q_WM[state, action] = reward
                elif reward == 0:
                    sub_list_corr.append(0)
                    q_RL[state, action] = q_RL[state, action] + gamma * alpha * (reward - q_RL[state, action])
                    q_WM[state, action] = q_WM[state, action] + gamma * (reward - q_WM[state, action])
                #print("\t\t\tdone updating q")
                q_WM = q_WM + phi * ((1/num_actions)-q_WM)
                #print("\t\t\tdone WM decay")

                # store data
                sub_list_sub_id.append(subj)
                sub_list_block_num.append(bl)
                sub_list_stim.append(state)
                sub_list_actions.append(action)
                sub_list_reward.append(reward)
                sub_list_stim_ctr.append(stim_counter[state])
                sub_list_set_size.append(set_size)
        #     print("\t\t -- end trial")
        # print("\t -- end block")

    sub_list_sub_id = np.array(sub_list_sub_id)
    sub_list_stim = np.array(sub_list_stim)
    sub_list_actions = np.array(sub_list_actions)
    sub_list_reward = np.array(sub_list_reward)
    sub_list_block_num = np.array(sub_list_block_num)
    sub_list_corr = np.array(sub_list_corr)
    sub_list_stim_ctr = np.array(sub_list_stim_ctr)

    sub_data = np.stack([sub_list_sub_id, sub_list_block_num, sub_list_stim, sub_list_actions, sub_list_reward, sub_list_corr, sub_list_stim_ctr, sub_list_set_size], axis=1)
    data = pd.DataFrame(sub_data, columns=['subj_idx', 'block_num', 'stim', 'response', 'feedback', 'corr', 'stim_ctr', 'set_size'])

    return data

In [6]:
model_rl = 'RLWM_v2'
config_rl = model_config_rl[model_rl]

In [7]:
s_size = [1, 2, 3, 4, 5, 6]
freq = [3, 4, 6, 3, 3, 3]

set_size_list = []
for i in range(len(s_size)):
    set_size_list.extend([s_size[i]]*freq[i])

np.random.shuffle(set_size_list)
set_size_list

[3, 3, 3, 5, 4, 3, 4, 2, 5, 6, 6, 1, 3, 2, 6, 2, 1, 2, 4, 5, 3, 1]

In [8]:
num_rep_per_block = [15]

num_datasets = 1
num_actions = 3
subjs = 20

In [9]:
priors_rl_alpha = [0, 0.01]
priors_rl_phi = [0, 0.40]
priors_rl_rho = [0.75, 1]
priors_rl_gamma = [0.60, 0.90]
priors_rl_epsilon = [0, 0.05]
priors_rl_C = [2, 5]

rl_alpha = np.random.uniform(priors_rl_alpha[0], priors_rl_alpha[1], subjs)
rl_phi = np.random.uniform(priors_rl_phi[0], priors_rl_phi[1], subjs)
rl_rho = np.random.uniform(priors_rl_rho[0], priors_rl_rho[1], subjs)
rl_gamma = np.random.uniform(priors_rl_gamma[0], priors_rl_gamma[1], subjs)
rl_epsilon = np.random.uniform(priors_rl_epsilon[0], priors_rl_epsilon[1], subjs)
rl_C = np.random.uniform(priors_rl_C[0], priors_rl_C[1], subjs)

In [10]:
file = list()

for n in range(num_datasets):
    print("\n==> dataset: ", n)
    dataset_file = {}

    dataset_file['info'] = {
                        'num_datasets': num_datasets, 'num_subj': subjs, 'num_actions': num_actions,
                        'model_rl': model_rl,
                        'set_size_list': set_size_list, 'num_rep_per_block': num_rep_per_block
                        }

    dataset_file['data'] = list()
    for i in range(subjs):
        print("[dataset: %d] generating subj data: %d" % (n, i))

        subj_param_rl = np.array([rl_alpha[i], rl_phi[i], rl_rho[i], rl_gamma[i], rl_epsilon[i], rl_C[i]])

        subj_data = {}

        sim_data = simulate_RLWM(subj_param_rl, set_size_list, num_rep_per_block, num_actions=num_actions, subjs=1)
        sim_data['subj_idx'] = i

        subj_data['subj_idx'] = i
        subj_data['true_param'] = subj_param_rl
        subj_data['sim_data'] = sim_data

        #print("\t --", subj_param_rl, subj_data['true_param'])
        dataset_file['data'].append(subj_data)

    file.append(dataset_file)


==> dataset:  0
[dataset: 0] generating subj data: 0
[dataset: 0] generating subj data: 1
[dataset: 0] generating subj data: 2
[dataset: 0] generating subj data: 3
[dataset: 0] generating subj data: 4
[dataset: 0] generating subj data: 5
[dataset: 0] generating subj data: 6
[dataset: 0] generating subj data: 7
[dataset: 0] generating subj data: 8
[dataset: 0] generating subj data: 9
[dataset: 0] generating subj data: 10
[dataset: 0] generating subj data: 11
[dataset: 0] generating subj data: 12
[dataset: 0] generating subj data: 13
[dataset: 0] generating subj data: 14
[dataset: 0] generating subj data: 15
[dataset: 0] generating subj data: 16
[dataset: 0] generating subj data: 17
[dataset: 0] generating subj data: 18
[dataset: 0] generating subj data: 19


In [28]:
dataset = file[0]['data']

In [29]:
list_sub_data = list()
for itr in range(len(dataset)):
    data = dataset[itr]['sim_data']
    data['subj_idx'] = itr

    list_sub_data.append(data)

PR_data = pd.concat(list_sub_data, ignore_index=True)

In [30]:
PR_data

,subj_idx,block_num,stim,response,feedback,corr,stim_ctr,set_size
0,0,0.0,1.0,0.0,0.0,0.0,1.0,3.0
1,0,0.0,2.0,0.0,1.0,1.0,1.0,3.0
2,0,0.0,0.0,1.0,0.0,0.0,1.0,3.0
3,0,0.0,2.0,2.0,0.0,0.0,2.0,3.0
4,0,0.0,2.0,2.0,0.0,0.0,3.0,3.0
...,...,...,...,...,...,...,...,...
22195,19,21.0,0.0,2.0,1.0,1.0,11.0,1.0
22196,19,21.0,0.0,2.0,1.0,1.0,12.0,1.0
22197,19,21.0,0.0,2.0,1.0,1.0,13.0,1.0
22198,19,21.0,0.0,2.0,1.0,1.0,14.0,1.0


###Reorganize data for vectorization

In [90]:
df = PR_data.sort_values(['subj_idx', 'block_num', 'stim'])

In [91]:
n_trials = df[['subj_idx','block_num','stim']].value_counts().max()
n_participants = df['subj_idx'].max() + 1
n_choices = int(df['response'].max()) + 1
n_blocks = int(df['block_num'].max()) + 1
n_stimuli = int(df['stim'].max()) + 1
shape = (n_trials, n_participants, n_choices, n_blocks, n_stimuli)
choices = np.ones(shape) * -np.inf
rewards = np.ones(shape) * -np.inf

for subj_idx in df['subj_idx'].unique():
    for block_num in df['block_num'].unique():
        subj_block_data = df[(df['subj_idx'] == subj_idx) & (df['block_num'] == block_num)]
        subj_block_choices = subj_block_data['response'].to_numpy(dtype=np.int32).reshape((-1, n_trials)).transpose((1,0))
        subj_block_rewards = subj_block_data['feedback'].to_numpy(dtype=np.float32).reshape((-1, n_trials)).transpose((1,0))
        n_stimuli = int(subj_block_data['set_size'].max())
        subj_block_choices = np.eye(n_choices)[subj_block_choices].transpose((0,2,1))
        subj_block_rewards = subj_block_rewards.reshape((n_trials, 1, n_stimuli)).repeat(n_choices, axis=1)
        choices[:, subj_idx, :, int(block_num), :n_stimuli] = subj_block_choices
        rewards[:, subj_idx, :, int(block_num), :n_stimuli] = subj_block_choices * subj_block_rewards